In [41]:
%pwd

'D:\\DS 01'

In [42]:
import os

os.chdir(r"D:\DS 01")  # Use raw string to handle spaces
print("Current directory:", os.getcwd())  # Verify the change


Current directory: D:\DS 01


### Data Ingestion Code

In [43]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir:Path



In [44]:
from src.DS01.constants import *
from src.DS01.utils.common import read_yaml,create_directories


In [45]:
class ConfigurationManager:
    def __init__(self,
                config_filepath=CONFIG_FILE_PATH,
                params_filepath=PARAMS_FILE_PATH,
                schema_filepath=SCHEMA_FILE_PATH):

        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [46]:
import urllib.request as request
import os
from src.DS01 import logger
import zipfile

In [47]:
## Data Ingestion Component

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    ## Downloading gthe Zip File
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info : \n {headers}")
        else:
            logger.info(f"{self.config.local_data_file} already exists ")


   ## Extracting Zip file
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
            
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)
           

In [48]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-16 15:16:09,912: INFO: common: yaml file :config\config.yaml loaded sucessfully]
[2025-02-16 15:16:09,915: INFO: common: yaml file :params.yaml loaded sucessfully]
[2025-02-16 15:16:09,918: INFO: common: yaml file :schema.yaml loaded sucessfully]
[2025-02-16 15:16:09,921: INFO: common: Directory created at artifacts]
[2025-02-16 15:16:09,923: INFO: common: Directory created at artifacts/data_ingestion]
[2025-02-16 15:16:10,330: INFO: 3123861799: artifacts/data_ingestion/data.zip downloaded with following info : 
 Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F48F:140B45:D585DB:E54639:67B1BA96
Accept-Ranges: bytes
Date: Sun, 16 F